# Bike Sharing Demand Forecasting Project   
0) Create virtual environment 
1) Part 1 - Data Ingestion from Azure Blob Storage to Bronze folder 
2) Part 2 - Data Transformation and Cleaning in Silver folder, then finally data is stored in Gold folder. 
3) Create Data Modelling and Visualization using PowerBI 
4) Leverage ML concepts to build prediction model ( EDA, Feature Selection, Model development, Evaluation, Deployment). 
5) Build MVP streamlit app and FastAPI (optional) for end users for predictions 

In [ ]:
# Check PySpark Version
import pyspark 
print("PySpark Version:", pyspark.__version__)

PySpark Version: 3.4.1


In [ ]:
# Simple Spark Session Test
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
spark.range(5).show()



+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
+---+



#  Bronze Layer Construction
1. Data Ingestion from Azure Blob Storage 

 a) Import libraries 
 b) Set Connect to Azure Blob Storage API to retieve the files from container 
 c) Validate Files exists in Blob Storage, check sizes and Schema of the files
 d) Load to bronze folder path

In [116]:
# import the necessary libraries 

from pyspark.sql import SparkSession
import os
from dotenv import load_dotenv
from azure.identity import ClientSecretCredential 
from azure.storage.blob import BlobServiceClient
from azure.keyvault.secrets import SecretClient 
from pyspark.sql.functions import lit, current_date
import pandas as pd 
import io
import time
from azure.core.exceptions import ResourceNotFoundError
from datetime import datetime
import pyarrow as pa
import pyarrow.parquet as pq

In [117]:
load_dotenv()

True

In [119]:
# Load environment variables
client_id=os.environ["AZURE_CLIENT_ID"]
tenant_id=os.environ["AZURE_TENANT_ID"] 
client_secret=os.environ["AZURE_CNT_SECRET"]
vault_url=os.environ["AZURE_VAULT_URL"]
container_name=os.environ["AZURE_CONTAINER_NAME"]
bronze_folder_path=os.environ["BRONZE_FOLDER_PATH"]
silver_folder_path=os.environ["SILVER_FOLDER_PATH"]
gold_folder_path=os.environ["GOLD_FOLDER_PATH"]

credentials = ClientSecretCredential(
    tenant_id=tenant_id,
    client_id=client_id,
    client_secret=client_secret 
) 

print("Connection to blob storage successfully.")

Connection to blob storage successfully.


In [75]:
# Create blob service client
blob_service_client = BlobServiceClient(
    account_url=vault_url,
    credential=credentials    # your ClientSecretCredential
)
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
def load_validated_csv(blob_name: str) -> pd.DataFrame:
    """
    Validates blob existence and size, then loads CSV into pandas.
    Prevents loading if blob is missing or empty.
    """
    # Step 1 — Validate existence and size
    try:
        blob_client = container_client.get_blob_client(blob_name)
        props = blob_client.get_blob_properties()
    except ResourceNotFoundError:
        raise FileNotFoundError(f"❌ Missing required file: {blob_name}")

    if props.size == 0:
        raise ValueError(f"❌ File '{blob_name}' exists but is EMPTY")

    print(f"✅ {blob_name} validated (size = {props.size} bytes)")

    # Step 2 — Load CSV into memory
    data = blob_client.download_blob().readall()
    df = pd.read_csv(io.BytesIO(data))

    print(f"📄 Loaded {blob_name}: {df.shape[0]} rows, {df.shape[1]} columns")
    return df
        

In [ ]:
# List of required file
bikesharing_df = load_validated_csv("bikesharing.csv")
dates_df       = load_validated_csv("dates.csv")

✅ bikesharing.csv validated (size = 1151577 bytes)
📄 Loaded bikesharing.csv: 17379 rows, 17 columns
✅ dates.csv validated (size = 52302 bytes)
📄 Loaded dates.csv: 1046 rows, 9 columns


In [ ]:
# Check schema of both the files

expected_bikesharing_schema = dict(zip(bikesharing_df.columns, bikesharing_df.dtypes.astype(str)))
print(expected_bikesharing_schema)

expected_dates_schema = dict(zip(dates_df.columns, dates_df.dtypes.astype(str)))
print(expected_dates_schema)

{'date': 'object',
 'datekey': 'int64',
 'day': 'int64',
 'month': 'int64',
 'month_name': 'object',
 'year': 'int64',
 'weekday': 'object',
 'weekofyear': 'int64',
 'is_weekend': 'int64'}

In [96]:
def validate_schema(df: pd.DataFrame, expected_schema: dict, file_name: str):
    """
    Validate that the dataframe matches the expected schema:
    - Same columns
    - Same dtypes
    """
    # Check columns match
    df_cols = list(df.columns)
    expected_cols = list(expected_schema.keys())

    if df_cols != expected_cols:
        raise ValueError(
            f"❌ Schema mismatch in {file_name}\n"
            f"Expected columns: {expected_cols}\n"
            f"Found columns:    {df_cols}"
        )

    # Check data types match
    for col, expected_type in expected_schema.items():
        actual_type = str(df[col].dtype)

        if actual_type != expected_type:
            raise ValueError(
                f"❌ Datatype mismatch in {file_name} for column '{col}'\n"
                f"Expected: {expected_type}, Found: {actual_type}"
            )

    print(f"✅ Schema validated successfully for {file_name}")


In [ ]:
# Validate schemas from function
validate_schema(bikesharing_df, expected_bikesharing_schema, "bikesharing.csv")
validate_schema(dates_df, expected_dates_schema, "dates.csv")

✅ Schema validated successfully for bikesharing.csv
✅ Schema validated successfully for dates.csv


In [102]:
# Add data extraction timestamp column for both dataframes and save to bronze folder

current_ts = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
bikesharing_df["data_extract_ts"] = current_ts
dates_df["data_extract_ts"] = current_ts

In [103]:
# Review the final dataframes   
display(bikesharing_df.head())
display(dates_df.head())

,index,date,season,year,month,hour,holiday,weekday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,demand,data_extract_ts
0,1,1/01/2023,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,2025-11-14 00:35:12
1,2,1/01/2023,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,2025-11-14 00:35:12
2,3,1/01/2023,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,2025-11-14 00:35:12
3,4,1/01/2023,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,2025-11-14 00:35:12
4,5,1/01/2023,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,2025-11-14 00:35:12


,date,datekey,day,month,month_name,year,weekday,weekofyear,is_weekend,data_extract_ts
0,2023-01-01,20230101,1,1,January,2023,Sunday,52,1,2025-11-14 00:35:12
1,2023-01-02,20230102,2,1,January,2023,Monday,1,0,2025-11-14 00:35:12
2,2023-01-03,20230103,3,1,January,2023,Tuesday,1,0,2025-11-14 00:35:12
3,2023-01-04,20230104,4,1,January,2023,Wednesday,1,0,2025-11-14 00:35:12
4,2023-01-05,20230105,5,1,January,2023,Thursday,1,0,2025-11-14 00:35:12


In [129]:
# save dataframes as parquet files locally and upload to bronze folder in local directory

def save_parquet_local(df, file_path):
    table = pa.Table.from_pandas(df)
    pq.write_table(table, file_path)
    rel_path = os.path.relpath(file_path)
    print(f"Saved to local file directory: {rel_path}")


In [130]:
save_parquet_local(bikesharing_df, bronze_folder_path + "/bikesharing.parquet")
save_parquet_local(dates_df,       bronze_folder_path + "/dates.parquet") 
print("Data saved to bronze folder successfully!!!!")

Saved to local file directory: ..\business_intelligence\data\1.bronze\bikesharing.parquet
Saved to local file directory: ..\business_intelligence\data\1.bronze\dates.parquet
Data saved to bronze folder successfully!!!!


# Silver Layer Construction  
2) Data Tranformation and Cleaning 

In [ ]:
# Extract and read the parquet file from bronze folder to validate 

df = pd.read_parquet(bronze_folder_path + "/bikesharing.parquet")
df.head()

,index,date,season,year,month,hour,holiday,weekday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,demand,data_extract_ts
0,1,1/01/2023,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,2025-11-14 00:35:12
1,2,1/01/2023,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,2025-11-14 00:35:12
2,3,1/01/2023,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,2025-11-14 00:35:12
3,4,1/01/2023,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,2025-11-14 00:35:12
4,5,1/01/2023,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,2025-11-14 00:35:12
